In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('Resources/train.csv')  # Replace with your local path if needed
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
# Extract title from name
df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

# Create FamilySize feature
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1

<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
C:\Users\the_a\AppData\Local\Temp\ipykernel_22652\839286812.py:2: SyntaxWarning: invalid escape sequence '\.'
  df['Title'] = df['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

# Fill missing values
df['Age'].fillna(df['Age'].median(), inplace=True)
df['Fare'].fillna(df['Fare'].median(), inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

# Encode categoricals
le = LabelEncoder()
df['Sex'] = le.fit_transform(df['Sex'])
df['Embarked'] = le.fit_transform(df['Embarked'])
df['Title'] = le.fit_transform(df['Title'])

# Normalize numeric columns
scaler = StandardScaler()
df[['Age', 'Fare', 'FamilySize']] = scaler.fit_transform(df[['Age', 'Fare', 'FamilySize']])

C:\Users\the_a\AppData\Local\Temp\ipykernel_22652\3254234926.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
C:\Users\the_a\AppData\Local\Temp\ipykernel_22652\3254234926.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [5]:
X = df.drop(columns='Survived')
y = df['Survived']
y_flipped = 1 - y  # "NotSurvived" instead of "Survived"

X_train, X_test, y_train, y_test = train_test_split(X, y_flipped, test_size=0.2, random_state=42)

In [6]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'K-Nearest Neighbors': KNeighborsClassifier(n_neighbors=3),
    'Decision Tree': DecisionTreeClassifier(max_depth=5, random_state=42)
}

model_results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    acc = accuracy_score(y_test, preds)
    model_results.append({'Model': name, 'Accuracy': round(acc * 100, 2)})
    print(f"📌 {name}\nAccuracy: {acc:.2f}")
    print(classification_report(y_test, preds))

📌 Logistic Regression
Accuracy: 0.80
              precision    recall  f1-score   support

           0       0.77      0.73      0.75        74
           1       0.82      0.85      0.83       105

    accuracy                           0.80       179
   macro avg       0.79      0.79      0.79       179
weighted avg       0.80      0.80      0.80       179

📌 K-Nearest Neighbors
Accuracy: 0.81
              precision    recall  f1-score   support

           0       0.80      0.72      0.76        74
           1       0.81      0.88      0.84       105

    accuracy                           0.81       179
   macro avg       0.81      0.80      0.80       179
weighted avg       0.81      0.81      0.81       179

📌 Decision Tree
Accuracy: 0.80
              precision    recall  f1-score   support

           0       0.83      0.65      0.73        74
           1       0.79      0.90      0.84       105

    accuracy                           0.80       179
   macro avg       0.81

In [7]:
# Use best model (example: Decision Tree)
best_model = models['Decision Tree']
y_pred = best_model.predict(X_test)

# Revert flipped labels to real Survived status
real_y_test = 1 - y_test

# Predicted NotSurvived but actually Survived = unexpected beneficiaries
predicted_not_survived = y_pred == 1
actually_survived = real_y_test == 1
beneficiaries_index = X_test[predicted_not_survived & actually_survived].index

beneficiaries_df = df.loc[beneficiaries_index]
beneficiaries_df[['Pclass', 'Sex', 'Age', 'Fare', 'FamilySize', 'Survived']].head()

,Pclass,Sex,Age,Fare,FamilySize,Survived
709,3,1,-0.104637,-0.341452,0.679295,1
621,1,1,0.971260,0.409741,0.059160,1
447,1,1,0.356462,-0.113846,-0.560975,1
192,3,0,-0.796286,-0.490280,0.059160,1
673,2,1,0.125912,-0.386671,-0.560975,1


In [8]:
results_df = pd.DataFrame(model_results)
results_df.to_csv('model_results_flipped.csv', index=False)
beneficiaries_df.to_csv('beneficiary_survivors.csv', index=False)